# 🔧 BRK-A-01 Topology Optimization

Notebook interattivo per il caso pilota **BRK-A-01** - Staffa Aeronautica.

**Obiettivo:** Eseguire topology optimization SIMP 3D per minimizzare la compliance
con vincolo di volume al 25%.

---

## 1. Setup e Import

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Moduli progetto
from src.geometry import create_bracket_domain, VoxelDomain
from src.loads import create_brk_a_01_static_case_1, LoadCase
from src.fem import MaterialProperties
from src.topopt import SIMPOptimizer, SIMPParams, TOResult

print("Imports completati ✓")

## 2. Creazione Dominio Voxel

Creiamo la griglia 3D con le zone:
- **Fixed (1):** Base e occhiello
- **Design (0):** Volume ottimizzabile
- **Void (-1):** Fori

In [ ]:
# Parametri dominio
# NOTA: Per test rapidi, usare risoluzione più bassa (es. 5mm)
RESOLUTION_MM = 5.0  # mm (usa 1.0 per produzione)

# Crea dominio
domain = create_bracket_domain(
    size_mm=(120, 60, 80),
    resolution_mm=RESOLUTION_MM
)

print(f"📐 Dominio creato:")
print(f"   Shape: {domain.shape}")
print(f"   Size: {domain.size_mm} mm")
print(f"   Elementi totali: {domain.n_elements:,}")
print(f"   Design space: {domain.n_design:,} elementi")
print(f"   Fixed: {domain.n_fixed:,} elementi")

In [ ]:
# Visualizza slice del dominio
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Slice a diversi livelli Z
z_levels = [2, domain.shape[2]//2, domain.shape[2]-5]
titles = ['Base (Z=2)', 'Centro', 'Top (occhiello)']

cmap = plt.cm.colors.ListedColormap(['white', 'lightblue', 'darkblue'])
bounds = [-1.5, -0.5, 0.5, 1.5]
norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)

for ax, z, title in zip(axes, z_levels, titles):
    im = ax.imshow(domain.grid[:, :, z].T, cmap=cmap, norm=norm, origin='lower')
    ax.set_title(f'{title} (Z={z})')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')

# Colorbar
cbar = plt.colorbar(im, ax=axes, ticks=[-1, 0, 1], shrink=0.8)
cbar.ax.set_yticklabels(['Void', 'Design', 'Fixed'])

plt.tight_layout()
plt.show()

## 3. Definizione Carichi

**Caso Statico 1:** Carico verticale 15 kN sull'occhiello, base vincolata.

In [ ]:
# Crea load case
load_case = create_brk_a_01_static_case_1(
    domain_shape=domain.shape,
    resolution_mm=RESOLUTION_MM
)

print(f"📌 Load Case: {load_case.name}")
print(f"   {load_case.description}")
print(f"   Nodi vincolati (base): {load_case.boundary_conditions[0].n_nodes}")

if load_case.distributed_loads:
    dl = load_case.distributed_loads[0]
    print(f"   Carico distribuito su {len(dl.node_indices)} nodi")
    print(f"   Forza totale: {dl.total_force} N")

## 4. Proprietà Materiale

**Ti6Al4V** - Lega di titanio per applicazioni aerospaziali.

In [ ]:
material = MaterialProperties(
    E=113.8e9,    # 113.8 GPa
    nu=0.342,
    rho=4430.0    # kg/m³
)

print(f"🔩 Materiale: Ti6Al4V")
print(f"   E = {material.E/1e9:.1f} GPa")
print(f"   ν = {material.nu}")
print(f"   ρ = {material.rho} kg/m³")

## 5. Parametri Ottimizzazione SIMP

In [ ]:
params = SIMPParams(
    volume_fraction=0.25,    # 25% volume target
    penalty=3.0,             # Penalità SIMP
    filter_radius=2.0,       # Raggio filtro [elementi]
    move_limit=0.2,          # Max variazione per iter
    max_iterations=50,       # Ridotto per test
    convergence_tol=0.01
)

print(f"⚙️ Parametri SIMP:")
print(f"   Volume target: {params.volume_fraction:.0%}")
print(f"   Penalità: {params.penalty}")
print(f"   Filtro: {params.filter_radius} elementi")
print(f"   Max iterazioni: {params.max_iterations}")

## 6. Esecuzione Topology Optimization

⚠️ **Attenzione:** Con griglia fine (1mm) il calcolo può richiedere diversi minuti.

In [ ]:
# Crea ottimizzatore
optimizer = SIMPOptimizer(
    domain=domain,
    load_cases=[load_case],
    material=material,
    params=params,
    verbose=True
)

# Esegui ottimizzazione
print("\n🚀 Avvio ottimizzazione...\n")
result = optimizer.run()

## 7. Analisi Risultati

In [ ]:
# Plot convergenza
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Compliance
ax1.plot(result.compliance_history, 'b-', linewidth=2)
ax1.set_xlabel('Iterazione')
ax1.set_ylabel('Compliance [J]')
ax1.set_title('Convergenza Compliance')
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Volume
ax2.plot(result.volume_history, 'r-', linewidth=2)
ax2.axhline(y=params.volume_fraction, color='k', linestyle='--', label='Target')
ax2.set_xlabel('Iterazione')
ax2.set_ylabel('Volume Fraction')
ax2.set_title('Convergenza Volume')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\n📊 Risultati finali:")
print(f"   Iterazioni: {result.iterations}")
print(f"   Converso: {result.converged}")
print(f"   Compliance finale: {result.final_compliance:.4e}")
print(f"   Volume finale: {result.final_volume:.4f}")
print(f"   Tempo: {result.elapsed_time:.1f}s")

In [ ]:
# Visualizza campo di densità - Slice
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Slice XY a diversi Z
z_levels = [2, domain.shape[2]//3, domain.shape[2]//2, 
            2*domain.shape[2]//3, domain.shape[2]-5, domain.shape[2]-2]

for ax, z in zip(axes.flat, z_levels):
    im = ax.imshow(result.density[:, :, z].T, cmap='gray_r', 
                   vmin=0, vmax=1, origin='lower')
    ax.set_title(f'Z = {z}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')

plt.colorbar(im, ax=axes, label='Densità', shrink=0.8)
plt.suptitle('Campo di Densità - Slice XY', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizzazione 3D semplificata
# (per visualizzazione avanzata usare PyVista)

threshold = 0.5
solid = result.density >= threshold

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Trova voxel solidi
x, y, z = np.where(solid)

# Subsample per performance
if len(x) > 5000:
    idx = np.random.choice(len(x), 5000, replace=False)
    x, y, z = x[idx], y[idx], z[idx]

ax.scatter(x, y, z, c=result.density[x, y, z], cmap='viridis', 
           s=20, alpha=0.6)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title(f'Topologia Ottimizzata (threshold={threshold})')

plt.tight_layout()
plt.show()

print(f"\n🎯 Voxel solidi (ρ ≥ {threshold}): {np.sum(solid):,}")
print(f"   Percentuale: {np.sum(solid)/domain.n_design*100:.1f}% del design space")

## 8. Export Risultati

In [ ]:
# Salva campo di densità
output_path = '../data/brk_a_01/density_field.npy'
np.save(output_path, result.density)
print(f"💾 Campo densità salvato in: {output_path}")

# Salva metadati
metadata = {
    'case': 'BRK-A-01',
    'resolution_mm': RESOLUTION_MM,
    'shape': domain.shape,
    'volume_fraction_target': params.volume_fraction,
    'volume_fraction_final': result.final_volume,
    'compliance_final': result.final_compliance,
    'iterations': result.iterations,
    'converged': result.converged
}

import json
with open('../data/brk_a_01/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print("💾 Metadati salvati")

---

## 📝 Prossimi Passi

1. **Aumentare risoluzione** a 1mm per risultato finale
2. **Aggiungere vincoli DfAM:** overhang, minimum member size
3. **Multi-load case:** includere Caso Statico 2
4. **Export STL:** mesh del risultato per stampa
5. **Validazione FEM:** verifica stress con mesh fine